# Circuitron Prototype

This notebook serves as the initial prototype for building **Circuitron** utilizing the OpenAI Agents SDK.

## Requirements

In [1]:
"""requirements for this prototype"""
# openai-agents
# logfire
# python-dotenv
# skidl>=2.0.1
# Node tool for SKiDL SVG export: npm install -g netlistsvg@1.0.2

'requirements for this prototype'

## Load the environment variables from .env file

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Set up tracing

In [3]:
import logfire

logfire.configure()
logfire.instrument_openai_agents()

Logfire project URL: https://logfire-us.pydantic.dev/shaurya-sethi/circuitron


# Instructions for the agents

In [4]:
"""Refactored prompts for structured outputs - no format instructions needed."""

# ---------- Planning Agent Prompt ----------
PLAN_PROMPT = """You are Circuitron-Planner, an expert PCB designer.

Analyze the user's requirements and create a comprehensive design solution.

Your analysis should include:

1. **Schematic Overview**: Break down the design into high-level functional blocks
2. **Calculations**: Document all design assumptions, equations, and derivations. 
   - Instead of computing numerical results directly, **output the calculation as clear, executable Python code using only standard math libraries** (e.g., `v = 10; i = 5; r = v/i; print(r)`). 
   - When a result is needed, **write code to perform the calculation** and request that it be executed using the provided calculation tool to obtain accurate values.
3. **Actions**: List specific implementation steps in order
4. **Component Search**: Identify and list all components needed for the design.
5. **Implementation Notes**: Provide SKiDL-specific guidance for later stages
6. **Limitations**: Note any missing specifications or open questions

For component search queries, use SKiDL keyword style:
- Generic part types with specifications
- No numeric values for passives, keep model names for ICs  
- Examples: "opamp lm324 quad", "capacitor ceramic", "mosfet n-channel"

Focus on creating a complete, actionable plan that later agents can execute. **When calculations are required, always write them as code, not as internal reasoning or estimates.**
"""

# ---------- Part Search Agent Prompt ----------
PART_SEARCH_PROMPT = """
You are Circuitron-PartFinder, an expert in SKiDL component searches.

Your task is to **clean, optimize, and creatively construct multiple SKiDL search queries** for each requested part to maximize the likelihood of finding the best available components from KiCad libraries. You are not limited to a single query: use multiple approaches in sequence, from broad to highly specific, and exploit all SKiDL search features as shown in the official documentation.

**SKiDL Search Query Construction Rules:**
- Output a *ranked list* of SKiDL search queries for each part, ordered from most general to most specific.
- Start with a broad/general query using only lowercase keywords (e.g., "opamp").
- Add more specific queries that include distinguishing features or model numbers (e.g., "opamp lm324 quad").
- For ICs/semiconductors: always try an exact model number regex anchor (e.g., "^lm324$").
- Use quoted phrases for features that are commonly described together in the library ("high performance", "precision low noise").
- Use the `|` (or) operator if searching for multiple common variants or packages is likely to help (e.g., "opamp (dip-8|soic-8)").
- Remove all numbers, units, and packages from passives ("capacitor 1uF 0603" → "capacitor").
- Remove duplicate terms while preserving logical order.
- Separate all tokens with single spaces.
- Prefer to output 2–5 queries per part (general → specific). If a part is very well-known or likely to have a unique identifier, include an exact-match query using regex anchors.

**Guidance & Features (from official SKiDL documentation):**
- SKiDL's `search()` finds parts matching **all** provided terms, anywhere in name, description, or keywords.
- Quoted strings match exact phrases.
- Regex anchors (`^model$`) return only parts with that exact name.
- The `|` operator matches parts containing at least one of the options.
- Use multiple search styles to maximize chances of finding the correct part, as libraries vary in their naming.

**Examples:**
- *User query*: "capacitor 0.1uF 0603"
    - "capacitor"
- *User query*: "opamp lm324 quad"
    - "opamp"
    - "opamp lm324"
    - "^lm324$"
- *User query*: "opamp low-noise dip-8"
    - "opamp"
    - "opamp low-noise"
    - "opamp dip-8"
    - "opamp (low-noise|dip-8)"
    - (if relevant) "opamp \"low noise\""
- *User query*: "mosfet irf540n to-220"
    - "mosfet"
    - "mosfet irf540n"
    - "^irf540n$"
    - "mosfet to-220"
    - "mosfet (to-220|d2pak)"

**After constructing the queries you have access to a tool to execute the queries to find the required parts - please make use of it.** 
"""

# ---------- Part Selection Agent Prompt ----------
PART_SELECTION_PROMPT = """
You are Circuitron-PartSelector, an expert in KiCad component selection.

Your task is to select the most optimal component(s) from a list of candidates returned by SKiDL's search() function, taking into account the provided design plan and technical requirements.

**How to select the best part(s):**
- Carefully review the design plan and requirements, including any electrical specs, key features, preferred packages, or application context.
- Analyze the list of available parts, considering:
    - Electrical/functional suitability: Does the part meet or exceed the technical needs?
    - Package type: Does it match any preferences, constraints, or standard footprints for the design?
    - Part/model specificity: If a model is requested (e.g., "LM324"), prioritize exact matches, but consider close or drop-in alternatives if a perfect match is missing.
    - Availability and practicality: Prefer common, widely-available, or manufacturer-supported parts.
    - Performance and modernity: Where specs are otherwise equal, choose higher-performance or more modern parts, unless compatibility with legacy parts is required.
    - Library source: If possible, prefer official or well-maintained libraries.
- Use clear, reasoned logic for your choice, weighing tradeoffs as an experienced engineer would.

If multiple parts are equally suitable (identical in all practical respects), you may select a shortlist, but always clarify your reasoning.

Your output should reflect the optimal component choice(s) and your rationale, leveraging all information given about the design and available options.
"""


# ---------- Documentation Agent Prompt ----------
DOC_AGENT_PROMPT = """You are Circuitron-DocSeeker, preparing for SKiDL code generation.

Based on the design plan and selected components, identify the specific SKiDL API questions that need research before coding.

Focus on:
- Part instantiation syntax for the specific components
- Pin connection methods and naming conventions
- Required setup calls (ERC, generate_svg, etc.)
- Power rail configuration
- Library-specific requirements

Prioritize questions as high/medium/low based on code generation impact.

Examples of good questions:
- "How to instantiate LM324 quad opamp in SKiDL?"
- "What is the pin naming convention for SOT-23 MOSFETs?"
- "How to set up VCC and GND power rails properly?"

Generate focused, actionable research queries.
Use MCP tools to find relevant SKiDL documentation and examples.
Focus on gathering the information needed to generate complete, executable SKiDL code.
"""

# ---------- Code Generation Agent Prompt ----------
CODE_GENERATION_PROMPT = """You are Circuitron-Coder, a SKiDL specialist.

Generate complete, executable SKiDL code based on the design plan, selected components and all provided official SKiDL documentation, usage examples, and API references.
Requirements:
1. Use ONLY the approved components from the parts list
2. Instantiate parts with proper library:name format
3. Create named power rails (VCC, GND) with appropriate settings
4. Connect all components according to the design plan
5. Include ERC() call for error checking
6. Include generate_svg("schematic.svg") for visualization
7. Follow SKiDL best practices

Use MCP tools if you need additional SKiDL documentation or examples.

Generate production-ready code that will execute without errors.
"""

# ---------- Code Execution Agent Prompt ----------
CODE_EXECUTION_PROMPT = """
You are Circuitron-Executor, a SKiDL code execution specialist.

Your role is to ensure that all generated SKiDL code is fully accurate and safe for execution.

**Instructions:**
1. Upon receiving generated SKiDL code, first utilize the available hallucination checking tool to verify the code against the official SKiDL knowledge graph. 
    - Ensure that all used classes, functions, attributes, and features are valid, properly named, and consistent with the SKiDL API and documentation.
    - Check that no hallucinated, deprecated, or non-existent SKiDL features are present.
2. If the code passes the hallucination check, proceed to execute the code using the provided execution tool and capture all outputs, results, and errors.
3. If the code fails the hallucination check or any inconsistencies are found, immediately flag the code as inaccurate, do not execute it, and pass it to the next agent for correction and refinement.
4. Always provide a clear summary of the verification outcome (pass/fail) and the execution results (or reasons for deferral).

Your goal is to ensure only valid, fully SKiDL-compliant code is executed, maintaining the highest standard of accuracy and reliability throughout the Circuitron pipeline.
"""

# ---------- Code Correction Agent Prompt ----------
CODE_CORRECTOR_PROMPT = """
You are SKiDL-Corrector, a specialist in identifying and fixing issues in SKiDL code.

Your responsibilities:
1. Carefully review the provided SKiDL code and the flagged errors or hallucinations identified by the knowledge graph and verification tools.
2. Utilize the MCP tool to retrieve the most relevant and correct official SKiDL documentation, API references, and usage examples needed to resolve all issues.
3. Correct all hallucinations, incorrect API usage, invalid functions, and any inconsistencies so that the code strictly follows SKiDL’s actual features and syntax.
4. Ensure the corrected code is fully executable, uses only real SKiDL constructs, and follows best practices as documented.
5. Once corrections are complete, hand off the updated code to Circuitron-Executor for another round of verification and execution.

Always rely on authoritative documentation and examples. Your goal is to ensure the code is fully SKiDL-compliant and production-ready before it moves to the execution stage.
Never alter or reinterpret the underlying design logic, methodology, or functional structure of the code. Only make corrections necessary to fix SKiDL-related errors, hallucinations, or API misuse. The fundamental design and engineering intent must remain unchanged.
In other words, no “creative rewrites” or design changes are allowed—the agent is limited to syntactic and semantic corrections that align with the original intent and structure.
"""



## Pydantic Basemodels for Structured Outputs

In [5]:
"""Pydantic BaseModels for structured outputs in the Circuitron pipeline.

This module defines all the Pydantic BaseModels required for getting structured outputs
from each agent in the pipeline, following OpenAI Agents SDK patterns for structured outputs.

Note: Models are simplified to use primitive types only to avoid JSON schema $defs issues
with the OpenAI Agents SDK strict validation.
"""

from enum import Enum
from typing import Any, Dict, List, Union
from pydantic import BaseModel, Field, ConfigDict


# ========== Planning Agent Models ==========

class PlanOutput(BaseModel):
    """Complete output from the Planning Agent."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    # Functional blocks as simple strings
    functional_blocks: List[str] = Field(default_factory=list, description="High-level functional blocks with name, description, inputs, and outputs as formatted strings")
    
    # Calculations as executable code strings
    calculation_codes: List[str] = Field(default_factory=list, description="All design calculations as executable Python code with descriptions")
    
    # Implementation actions as simple strings
    implementation_actions: List[str] = Field(default_factory=list, description="Ordered implementation steps with step numbers and dependencies")
    
    # Component search queries as simple strings
    component_search_queries: List[str] = Field(default_factory=list, description="SKiDL search queries for parts needed with component type and specifications")
    
    # Implementation notes as simple strings
    implementation_notes: List[str] = Field(default_factory=list, description="SKiDL-specific implementation guidance by category")
    
    # Limitations as simple strings
    design_limitations: List[str] = Field(default_factory=list, description="Missing specifications and open questions by category")

 
# ========== Part Search Agent Models ==========

class PartSearchOutput(BaseModel):
    """Complete output from the Part Search Agent."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    # Search plans as formatted strings
    search_plans: List[str] = Field(default_factory=list, description="Search plans for each component with ranked queries and rationales")
    
    # Search results as formatted strings  
    search_results: List[str] = Field(default_factory=list, description="Found parts with name, library, description, footprint, and specifications")

 
# ========== Part Selection Agent Models ==========

class PartSelectionOutput(BaseModel):
    """Complete output from the Part Selection Agent."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    # Selection criteria as formatted strings
    selection_criteria: List[str] = Field(default_factory=list, description="Electrical requirements, package preferences, and performance priorities")
    
    # Part evaluations as formatted strings
    part_evaluations: List[str] = Field(default_factory=list, description="Evaluation of all candidate parts with scores, pros, cons, and notes")
    
    # Selected parts as formatted strings
    selected_parts: List[str] = Field(default_factory=list, description="Final selected parts with rationale and alternatives")


# ========== Documentation Agent Models ==========

class PriorityLevel(str, Enum):
    """Priority levels for documentation queries."""
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"


class DocumentationOutput(BaseModel):
    """Complete output from the Documentation Agent."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    # Research queries as formatted strings
    research_queries: List[str] = Field(default_factory=list, description="Prioritized research queries with priority levels and context")
    
    # Documentation findings as formatted strings
    documentation_findings: List[str] = Field(default_factory=list, description="Research findings with code examples, API references, and best practices")
    
    # Implementation readiness assessment
    implementation_readiness: str = Field(..., description="Assessment of readiness for code generation")


# ========== Code Generation Agent Models ==========

class CodeGenerationOutput(BaseModel):
    """Complete output from the Code Generation Agent."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    # Complete SKiDL code
    complete_skidl_code: str = Field(..., description="Complete executable SKiDL code")
    
    # Code metadata as formatted strings
    imports: List[str] = Field(default_factory=list, description="Required import statements")
    power_rails: List[str] = Field(default_factory=list, description="Power rail configurations with names, voltages, and settings")
    components: List[str] = Field(default_factory=list, description="Component instantiations with names, parts, libraries, and parameters")
    connections: List[str] = Field(default_factory=list, description="All connections between components with from/to details and net names")
    validation_calls: List[str] = Field(default_factory=list, description="ERC and other validation calls")
    output_generation: List[str] = Field(default_factory=list, description="SVG and other output generation calls")
    
    # Implementation notes and assumptions
    implementation_notes: List[str] = Field(default_factory=list, description="Important notes about the implementation")
    assumptions: List[str] = Field(default_factory=list, description="Assumptions made during code generation")


# ========== Code Execution Agent Models ==========

class CodeExecutionOutput(BaseModel):
    """Complete output from the Code Execution Agent."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    # Hallucination check results
    hallucination_check_passed: bool = Field(..., description="Whether the code passed hallucination checks")
    hallucination_issues: List[str] = Field(default_factory=list, description="List of hallucination issues found")
    hallucination_confidence: float = Field(..., ge=0.0, le=1.0, description="Confidence score for the validation")
    
    # Execution results
    execution_success: bool = Field(..., description="Whether the code executed successfully")
    execution_output: str = Field(..., description="Output from code execution")
    execution_errors: List[str] = Field(default_factory=list, description="Any errors encountered during execution")
    execution_warnings: List[str] = Field(default_factory=list, description="Any warnings generated")
    files_generated: List[str] = Field(default_factory=list, description="List of files generated (e.g., SVG, netlist)")
    
    # Summary and next action
    verification_summary: str = Field(..., description="Summary of verification outcome")
    next_action: str = Field(..., description="Recommended next action (execute, correct, or flag)")


# ========== Code Correction Agent Models ==========

class CodeCorrectionOutput(BaseModel):
    """Complete output from the Code Correction Agent."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    # Issues and corrections as formatted strings
    issues_identified: List[str] = Field(default_factory=list, description="All issues identified with type, description, location, and severity")
    corrections_made: List[str] = Field(default_factory=list, description="All corrections applied with issue addressed, original code, corrected code, and rationale")
    documentation_references: List[str] = Field(default_factory=list, description="Documentation references used with source, section, and application")
    
    # Final corrected code
    corrected_code: str = Field(..., description="Complete corrected SKiDL code")
    validation_notes: str = Field(..., description="Notes about the validation and correction process")

 
# ========== Calculation Execution Models ==========

class CalculationRequest(BaseModel):
    """Request to execute a calculation."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    calculation_id: str = Field(..., description="Unique identifier for the calculation")
    description: str = Field(..., description="Description of what the calculation does")
    code: str = Field(..., description="Python code to execute")

 
class CalculationResult(BaseModel):
    """Result from executing a calculation."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    calculation_id: str = Field(..., description="Identifier of the executed calculation")
    success: bool = Field(..., description="Whether the calculation executed successfully")
    result_value: str = Field(..., description="Calculation result as a string representation")
    output: str = Field(..., description="Text output from the calculation")
    error: str = Field(default="", description="Error message if calculation failed")
    

class CalcResult(BaseModel):
    calculation_id: str
    success: bool
    stdout: str = ""
    stderr: str = ""


# ========== Common Models ==========

class AgentResponse(BaseModel):
    """Base model for agent responses with metadata."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    agent_name: str = Field(..., description="Name of the agent providing the response")
    timestamp: str = Field(..., description="Timestamp of the response")
    success: bool = Field(..., description="Whether the agent operation was successful")
    message: str = Field(..., description="Human-readable message about the operation")

 
class PipelineStatus(BaseModel):
    """Status of the overall pipeline execution."""
    model_config = ConfigDict(extra="forbid", strict=True)
    
    current_stage: str = Field(..., description="Current stage in the pipeline")
    completed_stages: List[str] = Field(default_factory=list, description="List of completed stages")
    pending_stages: List[str] = Field(default_factory=list, description="List of remaining stages")
    overall_progress: float = Field(..., ge=0.0, le=1.0, description="Overall progress as a percentage")
    status_message: str = Field(..., description="Current status description")


## Agents

In [7]:
from agents import function_tool
import subprocess, textwrap

@function_tool
async def execute_calculation(
    calculation_id: str,
    description: str,
    code: str,
) -> CalcResult:
    """
    Execute pure-Python maths code *generated by the LLM* in an isolated Docker container.

    Args:
        calculation_id: Correlates this request to its tool response.
        description: What this calculation is intended to compute.
        code: Python source (generated by the LLM) that prints the final value.
    Returns:
        CalcResult with stdout, stderr, and success flag.
    """
    safe_code = textwrap.dedent(code)
    docker_cmd = [
        "docker", "run", "--rm",
        "--network", "none",
        "--memory", "128m",
        "--pids-limit", "64",
        "python:3.12-slim",
        "python", "-c", safe_code,
    ]
    try:
        proc = subprocess.run(docker_cmd, capture_output=True, text=True, timeout=15)
        return CalcResult(
            calculation_id=calculation_id,
            success=proc.returncode == 0,
            stdout=proc.stdout.strip(),
            stderr=proc.stderr.strip(),
        )
    except Exception as e:
        return CalcResult(
            calculation_id=calculation_id,
            success=False,
            stderr=str(e),
        )

import asyncio
from agents import Agent, Runner

planner = Agent(
    name="Circuitron-Planner",
    instructions=PLAN_PROMPT,
    model="o4-mini",
    output_type=PlanOutput,
    tools=[execute_calculation],
)


user_prompt = "Design a non-inverting op-amp voltage follower using an LM324."
result = asyncio.run(Runner.run(planner, user_prompt))
print(result.final_output)

RuntimeError: asyncio.run() cannot be called from a running event loop